In [ ]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={
    "param1":"None1",
    "param2":"None2"}

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface()

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
#4 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

In [ ]:
log_name = "my_experiment_1"

tensorboard_log_dir, torch_writer_dir = substep.prepare_tensorbord_logs(log_name, copy_prev_logs=True)

In [ ]:
help(substep.prepare_tensorbord_logs)

In [ ]:
import random

import torch
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir=torch_writer_dir)

In [ ]:
# gen random lr, for all runs

lr = round(random.random(), 2)
print(f"{lr=}")

In [ ]:
x = torch.arange(-5, 5, 0.1).view(-1, 1)
y = -5 * x + 0.1 * torch.randn(x.size())

model = torch.nn.Linear(1, 1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)


def train_model(iter):
    for epoch in range(iter):
        y1 = model(x)
        loss = criterion(y1, y)
        writer.add_scalar("train/loss", loss, epoch)
        writer.add_scalar("train/lr", lr, epoch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


train_model(10)
writer.flush()

In [ ]:
%tensorboard --logdir $tensorboard_log_dir

In [ ]:
#8 stop spark
SinaraSpark.stop_session()